In [34]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import re

In [35]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [69]:
file_name = 'data.csv'
headers = ['Price', 'Location', 'Area', 'Street', 'Facade', 'Direction', 'Saler', 'Type', 'Floor', 'Bathrooms', 'Bedrooms', 'Legal Doc']
f = open(file_name, "w")   
f.write('\t'.join(headers))
f.write('\n')

1

In [70]:
source= f'https://meeyland.com'

In [ ]:
def get_info(html_text):
    info = []
    
    price = html_text.find('div', {'class':'article-price'}).string.strip('\n         ')
    location = html_text.find('div', {'class':'article-info-location'}).string.strip('\n         ')
    details = html_text.find_all('div', {'class':'col-6'})
    details = [detail.text.strip('\n\n               ') for detail in details]
    info.append(price)
    info.append(location)
    info.extend(details)

    link = source + html_text.find('a', {'class':"call-traking"}).attrs['href']
    time.sleep(1)
    new_html_text = requests.get(link).text
    new_tree = BeautifulSoup(new_html_text, 'html.parser')
    items = new_tree.find_all('li', {'class':'item'})
    more_info = ['---']*4

    for i in items:
        l = i.text
        if 'Số phòng ngủ' in l and '---' not in l:
            more_info[2] = re.findall(r'\d+', l)[0]
        if 'Số phòng tắm' in l and '---' not in l:
            more_info[1] = re.findall(r'\d+', l)[0]
        if 'Tầng thứ' in l and '---' not in l:
            # print(l)
            more_info[0] = re.findall(r'\d+', l)[0]
        if 'Giấy tờ pháp lý' in l:
            more_info[3] = l.split('\n')[2]

    info.extend(more_info)

    return info

page = 1
while True:
    # print(page)
    if page > 200:
        break

    try:
        url = f'https://meeyland.com/mua-ban-nha-dat/page-{page}'
        print(url)
        html_text = requests.get(url).text
        tree = BeautifulSoup(html_text, 'html.parser')
        results = tree.find_all('div', {'class':'col-sm-6 col-md-4'})

        for each in results:
            info = get_info(each)
            info = '\t'.join(info)
            f.write(info+'\n')
            
    except Exception as e:
        # print(f'Error: {e}')
        pass

    page += 1
    break


In [72]:
f.close()

In [73]:
df = pd.read_csv('data.csv', sep='\t')
df.head()

,Price,Location,Area,Street,Facade,Direction,Saler,Type,Floor,Bathrooms,Bedrooms,Legal Doc
0,4 Tỷ - 32.26 Triệu/m²,"Huyện Yên Phong, Tỉnh Bắc Ninh",124 m²,Ngõ 3 ô tô tránh,5.3m,"Nam, Bắc",Nhà môi giới,Nhà ở,---,---,12,Giấy CN QSDĐ - Sổ đỏ - Sổ hồng
1,4.5 Tỷ - 50 Triệu/m²,"Quận Tây Hồ, Thành phố Hà Nội",90 m²,Mặt phố - Mặt đường,---,---,Chính chủ,Chung cư,---,2,1,Giấy CN QSDĐ - Sổ đỏ - Sổ hồng
2,1.4 Tỷ - 14 Triệu/m²,"Thành phố Bảo Lộc, Tỉnh Lâm Đồng",100 m²,Ngõ 3 ô tô tránh,5m,Đông Bắc,Nhà môi giới,Đất - Đất nền - Nhà như đất,---,---,---,Giấy CN QSDĐ - Sổ đỏ - Sổ hồng
3,390 Triệu - 1.96 Triệu/m²,"Huyện Củ Chi, Thành phố Hồ Chí Minh",198.7 m²,Ngõ 1 ô tô,12.9m,Tây,Nhà môi giới,Đất - Đất nền - Nhà như đất,---,---,---,Giấy CN QSDĐ - Sổ đỏ - Sổ hồng
4,21.5 Tỷ - 3.5 Triệu/m²,"Huyện Cần Giờ, Thành phố Hồ Chí Minh","6,142 m²",Ngõ 4 ô tô trở lên,46m,Đông Nam,Nhà môi giới,Bất động sản khác,---,---,---,Giấy CN QSDĐ - Sổ đỏ - Sổ hồng


In [2]:
# url = f'https://meeyland.com/mua-ban-nha-dat/chinh-chu-cat-lo-sau-penstudio-90m2-tai-699-lac-long-quan-tay-ho-ha-noi-1624002772850'
# print(url)
# html_text = requests.get(url).text
# tree = BeautifulSoup(html_text, 'html.parser')
# items = tree.find_all('li', {'class':'item'})
# # more_info = [items[i].find('span',{'class':'attribute-info'}) for i in range(6,11, 2)]
# more_info = ['---']*4

# a = 'd'
# for i in items:
#     l = i.text
#     if 'Số phòng ngủ' in l and '---' not in l:
#         more_info[2] = re.findall(r'\d+', l)[0]
#     if 'Số phòng tắm' in l and '---' not in l:
#         more_info[1] = re.findall(r'\d+', l)[0]
#     if 'Tầng thứ' in l and '---' not in l:
#         # print(l)
#         more_info[0] = re.findall(r'\d+', l)[0]
#     if 'Giấy tờ pháp lý' in l:
#         print(l)
#         more_info[3] = l.split('\n')[2]
        
# print(more_info)

In [58]:
a.split('\n')[2]

'Giấy CN QSDĐ - Sổ đỏ - Sổ hồng'